In [16]:
import sys
import re
import pandas as pd
df_sample = pd.read_csv('dample.csv')

df_ph_csv = df_sample[['EID','Home Phone']]
df_ph_csv = df_ph_csv[df_ph_csv['EID'].notna()] # drops only rows where eid column is na
#df_ph_csv = df_ph_csv.dropna() #drops all rows that 'na' in any columns
#adding empty column
df_ph_csv['telephone'] = ''
#print(df_ph_csv)
raw_eid = df_ph_csv['EID'].dropna()
eid = raw_eid.tolist()
for i in range(len(df_ph_csv)) : 
    #df_ph_csv.iloc[i,2]=re.findall(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]',df_ph_csv.iloc[i,1])
    if pd.isnull(df_ph_csv.iloc[i,1]):
        continue
    df_ph_csv.iloc[i,2]=df_ph_csv.iloc[i,1].replace("(","").replace(")","").replace("-","").replace("/","")
    #print(df_ph_csv.iloc[i,2])
#print(df_ph_csv)

In [17]:
import cx_Oracle
connection = cx_Oracle.connect("xxxxx/xxxx@yyyyyys.com:9999/xxxxx.yyyy.com")
cursor = connection.cursor()
cursor.execute("select EID, phone_area_code, phone_nbr from cips_main m, cips_phone_info p where m.ALUMNI_ID = p.ALUMNI_ID AND m.ut_eid IN %s" % str(tuple(eid)).replace(',)',')'))
results = cursor.fetchall()
df_ph_db = results
cursor.close()
connection.close()
print(df_ph_db)
df_ph_db = pd.DataFrame(results, columns = ['EID','Area','phone'])
#dropping duplicates
df_ph_db = df_ph_db.drop_duplicates()
print(df_ph_db)

DatabaseError: ORA-12170: TNS:Connect timeout occurred

In [13]:
df_ph_db["telephone"] = df_ph_db["Area"].astype(str) + df_ph_db["phone"].astype(str)
#df_ph_db["telephone"] = df_ph_db["Area"] + df_ph_db["phone"]
print(df_ph_db)
#print(df_ph_db.iloc[:,3])

           EID Area    phone   telephone
0     0223RZWI  361  4632443  3614632443
1     0227VECQ  956  3305027  9563305027
3     0228CVDP  512  7316755  5127316755
4     0234IBFD  217  5774677  2175774677
5     0252RFCH  254  6981590  2546981590
...        ...  ...      ...         ...
2054     ZACKH  409  8325611  4098325611
2055     ZACKH  409  8383969  4098383969
2056     ZACKH  409  7824193  4097824193
2058     ZACKH  409  8962852  4098962852
2059    ZJS237  512  6956461  5126956461

[1996 rows x 4 columns]


In [1]:
# the phone number columns are string type and they are being converted to numeric 
#df_ph_db[['Area', 'phone','telephone']] = df_ph_db[['Area', 'phone','telephone']].apply(pd.to_numeric)
#print(df_ph_db.dtypes)

In [25]:
df_phone = pd.DataFrame()
#print(df_email)
for i in range(len(df_ph_csv)):
    id_csv = df_ph_csv.iloc[i,0]
    ph_csv = df_ph_csv.iloc[i,2]
    ph_db = (df_ph_db.loc[df_ph_db['EID'] == id_csv]).iloc[:,3]
    ph_db = {*ph_db}
    #print(ph_db, ph_csv)
    ph_csv = {ph_csv}
    diff = (ph_csv - ph_db)
    if len(diff) > 0:
            df_phone = df_phone.append({'EID':id_csv, 'telephone':df_ph_csv.iloc[i,2]}, ignore_index=True)

df_phone = df_phone.drop_duplicates()
#print(df_phone)
#df_phone.to_csv(r'C:\Users\le5752\Documents\DB_Python\ph.csv', index = False)

df_phone['DB_phone'] = ''
for i in range(len(df_phone)):
    id_csv = df_phone.iloc[i,0]
    ph_db = (df_ph_db.loc[df_ph_db['EID'] == id_csv]).iloc[:,3]
    #print(ph_db)
    df_phone.iloc[i,2] = ph_db.values.tolist()
    
df_phone = df_phone[df_phone['telephone'].notna()] 
df_phone['telephone'].dropna()
print(df_phone['telephone'].dropna())
#df_phone.to_csv(r'C:\Users\le5752\Documents\DB_Python\ph.csv', index = False)


0      512 5854209
1       3184581505
2       2146740564
3       5122930468
4       5126899957
          ...     
385     8172472483
386               
387               
388               
389               
Name: telephone, Length: 358, dtype: object
